# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

## We need some testing data

In [2]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [3]:
from computions import *

### get_describe_nominal function test

Basic situation

In [4]:
get_describe_nominal(test_frame.iloc[:,1], Y)

object_variable


,count,1.0,1.0%,0.0,0.0%,2.0,2.0%
object_variable,,,,,,,
1.0,128,26,20.312500,102.0,79.687500,0.0,0.000000
2.0,105,32,30.476190,73.0,69.523810,0.0,0.000000
4.0,103,73,70.873786,27.0,26.213592,3.0,2.912621
5.0,102,92,90.196078,8.0,7.843137,2.0,1.960784
9.0,99,46,46.464646,0.0,0.000000,53.0,53.535354
6.0,94,86,91.489362,2.0,2.127660,6.0,6.382979
3.0,90,45,50.000000,45.0,50.000000,0.0,0.000000
7.0,90,69,76.666667,1.0,1.111111,20.0,22.222222
8.0,88,60,68.181818,0.0,0.000000,28.0,31.818182


Data frame with emptys

In [5]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

object_variable
Is all obs. are in table -  False
object_variable
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [6]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{1.0: 0.543058776086299, 0.0: 0.2736606567779426, 2.0: 0.7792183132822943}
{'category 1': 0.543058776086299, 'cetegory 0': 0.2736606567779426, 'category 2': 0.7792183132822943}
{'category 1': 0.5473890980520543, 'cetegory 0': 0.27162192255350737, 'category 2': 0.7769752733282482}
{1.0: 0.7263393432220574, 0.0: 0.2736606567779426}


AUC nominal

In [7]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys')
print(get_AUC_numeric(na_containts_frame['object_variable'], Y_strs))

object_variable
object_variable
{'category 1': 0.8079604798870854, 'cetegory 0': 0.9246609000997142, 'category 2': 0.917479341353749}
AUC bynary
object_variable
{1.0: 0.9246609000997144, 0.0: 0.9246609000997142}
AUC with emptys
{'category 1': 0.6368762701331856, 'cetegory 0': 0.07536601239002241, 'category 2': 0.9178896871122492}


Get publish AUC for any predictor type

In [8]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

object_variable
nominal predictor without table
{1.0: {'AUC': 0.8079604798870854, 'rel_type': 1, 'GINI': 0.6159209597741708}, 0.0: {'AUC': 0.9246609000997142, 'rel_type': 1, 'GINI': 0.8493218001994285}, 2.0: {'AUC': 0.917479341353749, 'rel_type': 1, 'GINI': 0.834958682707498}}
moninal predictor with table
{'category 1': {'AUC': 0.8079604798870854, 'rel_type': 1, 'GINI': 0.6159209597741708}, 'cetegory 0': {'AUC': 0.9246609000997142, 'rel_type': 1, 'GINI': 0.8493218001994285}, 'category 2': {'AUC': 0.917479341353749, 'rel_type': 1, 'GINI': 0.834958682707498}}
numeric predictor
{1.0: {'AUC': 0.543058776086299, 'rel_type': 1, 'GINI': 0.08611755217259809}, 0.0: {'AUC': 0.7263393432220574, 'rel_type': -1, 'GINI': 0.4526786864441148}, 2.0: {'AUC': 0.7792183132822943, 'rel_type': 1, 'GINI': 0.5584366265645886}}


Decoding AUC info as DataFrame

In [9]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

1.0                          0.0                     \
                       AUC rel_type      GINI       AUC rel_type      GINI   
numeric_variable  0.543059        1  0.086118  0.726339       -1  0.452679   

                       2.0                     
                       AUC rel_type      GINI  
numeric_variable  0.779218        1  0.558437

Getting indicators from computions uotput

In [10]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


category 1                    cetegory 0                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.547389        1  0.094778   0.728378       -1  0.456756   

                 category 2                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.776975        1  0.553951     20        0.02

## Writing to excel

Adding a new sheet

In [11]:
xl_writer = pd.ExcelWriter("test_result/sheet_creation.xlsx",engine='xlsxwriter')

for i in range(20):
    add_sheet(xl_writer, 'some name')
    add_sheet(xl_writer, 'some very very very long long name')
xl_writer.close()

Adding a info about different predictors

In [12]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

object_variable


Let's show final table

In [13]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

object_variable


1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.543059        1  0.086118   
object_variable                                 0.80796        1  0.615921   
column with soo000oo000oo000oo000oo long name   0.53534        1  0.070681   

                                                    0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.726339       -1  0.452679   
object_variable                                0.924661        1  0.849322   
column with soo000oo000oo000oo000oo long name  0.533533       -1  0.067066   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.779218        1  0.558437   
object_variable                                0.917479        1  0.834959   
column with soo000oo000oo000oo000oo long name  0.513177       -1  0.026355   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [14]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

object_variable


Writing info about some predictor

In [15]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

object_variable


In [17]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()